# EDA 5번) 대구의 지형 특성 (분지) 분석
* 대구와 **비슷한** 분지 지형 기후 탐색
* 대구와 **다른 지형**의 지역 기후 탐색
* HI(열 지수)를 체감 온도로 두고 진행함
* 별도의 체감 온도 계산 식이 있을 경우 그걸로 진행하기로 결정
* 폭염 기준 : 주의보 체감 온도 33℃ 이상 / 경보 35℃ 이상
- 체감 온도: 기온에 습도, 바람 등의 영향이 더해져 사람이 느끼는 더위나 추위를 정량적으로 나타낸 온도
- 습도 10% 증가 시마다 체감온도 1도 가량 증가하는 특징

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_busan = pd.read_csv("output/Busan_pre.csv")
df_daegu = pd.read_csv("output/Daegu_pre.csv")
df_daejeon = pd.read_csv("output/Daejeon_pre.csv")
df_cheongju = pd.read_csv("output/Cheongju_pre.csv")
df_chuncheon = pd.read_csv("output/Chuncheon_pre.csv")

# 대구

* 특정 **연도**의 **월** 을 타겟으로 삼고 전체 HI와 DI의 평균
* 그래프로 비교
* 연도 변경 & 월 유지
* 연도 고정 & 월 변경
등 여러 방법으로...


In [3]:
daegu = df_daegu[['region','year','month','day','hour','rainfall','temp','lightning','humidity','wind_speed','wind_direction','condition','HI','DI']]
daegu

,region,year,month,day,hour,rainfall,temp,lightning,humidity,wind_speed,wind_direction,condition,HI,DI
0,동인동,2010,6,1,0,0.0,16.500000,-1,53.0,53.0,390.0,1,15.589444,60.743550
1,동인동,2010,6,1,1,0.0,18.100000,-1,49.0,53.0,390.0,1,17.245000,62.734310
2,동인동,2010,6,1,2,0.0,19.700001,-1,46.0,53.0,390.0,1,18.926668,64.650381
3,동인동,2010,6,1,3,0.0,20.799999,-1,41.0,53.0,390.0,1,20.006110,65.727719
4,동인동,2010,6,1,4,0.0,22.200001,-1,36.0,53.0,390.0,2,21.415557,67.046081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14831611,구지면,2022,4,30,19,0.0,10.700000,-1,71.0,2.1,335.0,-1,9.679444,52.335030
14831612,구지면,2022,4,30,20,0.0,11.000000,-1,65.0,2.2,339.0,-1,9.852778,52.993500
14831613,구지면,2022,4,30,21,0.0,11.900000,-1,61.0,2.2,338.0,-1,10.738333,54.402410
14831614,구지면,2022,4,30,22,0.0,13.400000,-1,53.0,2.0,351.0,-1,12.179444,56.605980


In [44]:
daegu[(daegu['year'] == 2013) & (daegu['HI'] >= 33) & (daegu['DI'] >= 60)]

,region,year,month,day,hour,rainfall,temp,lightning,humidity,wind_speed,wind_direction,condition,HI,DI
27052,동인동,2013,7,2,4,0.0,30.100000,0,65.0,1.3,77.0,4,34.099124,80.755350
27053,동인동,2013,7,2,5,0.0,30.400000,0,62.0,1.7,176.0,4,34.058703,80.717520
27054,동인동,2013,7,2,6,0.0,30.200001,0,60.0,2.6,177.0,4,33.217559,80.120801
27055,동인동,2013,7,2,7,0.0,30.600000,0,59.0,2.0,189.0,4,33.788028,80.522460
27056,동인동,2013,7,2,8,0.0,30.299999,0,62.0,2.1,235.0,4,33.850541,80.575139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14755640,구지면,2013,8,30,8,0.0,29.299999,0,69.0,1.3,252.0,4,33.194065,80.180829
14755973,구지면,2013,9,13,5,0.0,30.600000,0,59.0,3.2,185.0,2,33.788028,80.522460
14755974,구지면,2013,9,13,6,0.0,31.500000,0,58.0,3.5,198.0,2,35.394117,81.608300
14755975,구지면,2013,9,13,7,0.0,30.200001,0,63.0,4.2,184.0,3,33.864779,80.588741


In [43]:
daegu_target = daegu[(daegu['year'] == 2013) & (daegu['HI'] >= 33) & (daegu['DI'] >= 70)]
print(len(daegu_target['day'].unique()))

31


In [5]:
result = []

for i in range(2010,2022):
    daegu_sample = daegu[(daegu['year'] == i) & (daegu['temp'] >= 33)]
    result.append(len(daegu_sample['day'].unique()))

result

[29, 23, 26, 29, 18, 19, 30, 28, 31, 24, 25, 27]

In [6]:
sum = 0
for i in range(len(result)):
    sum += result[i]

mean = sum / len(result)
mean

25.75

In [32]:
result = []

for i in range(2010,2022):
    daegu_sample = daegu[(daegu['year'] == i) & (daegu['temp'] >= 33)]
    result.append(len(daegu_sample['day'].unique()))

print("대구 지역 매년 폭염 발생일 수" )
print(result)

sum = 0
for i in range(len(result)):
    sum += result[i]

mean = sum / len(result)


print("대구 지역 폭염 발생일 평균 : ",round(mean,2))

대구 지역 매년 폭염 발생일 수
[29, 23, 26, 29, 18, 19, 30, 28, 31, 24, 25, 27]
대구 지역 폭염 발생일 평균 :  25.75


* 폭염 건수를 Day 단위로 끊어서 Unique 하면 각 일자만 나오므로 폭염이 발생한 일 수 계산 가능
* 테스트 삼아 2017년 대구를 타겟으로 삼은 결과 : 28일

# 부산

In [9]:
busan = df_busan[['region','year','month','day','hour','rainfall','temp','lightning','humidity','wind_speed','wind_direction','condition','HI','DI']]
busan

,region,year,month,day,hour,rainfall,temp,lightning,humidity,wind_speed,wind_direction,condition,HI,DI
0,중앙동,2010,6,1,0,0.0,14.700000,-1,61.0,53.0,390.0,1,13.818333,58.361330
1,중앙동,2010,6,1,1,0.0,15.400000,-1,56.0,53.0,390.0,3,14.457778,59.303760
2,중앙동,2010,6,1,2,0.0,16.000000,-1,51.0,53.0,390.0,3,14.987222,60.045400
3,중앙동,2010,6,1,3,0.0,16.600000,-1,52.0,53.0,390.0,2,15.673333,60.855680
4,중앙동,2010,6,1,4,0.0,17.700001,-1,45.0,53.0,390.0,3,16.700557,62.087351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21411835,철마면,2022,4,30,19,0.0,11.700000,-1,63.0,2.5,359.0,-1,10.570556,54.065290
21411836,철마면,2022,4,30,20,0.0,11.400000,-1,61.0,2.5,340.0,-1,10.188333,53.695460
21411837,철마면,2022,4,30,21,0.0,12.100000,-1,56.0,1.1,345.0,-1,10.827778,54.801240
21411838,철마면,2022,4,30,22,0.0,14.300000,-1,45.0,1.4,307.0,-1,12.960556,57.818650


In [31]:
result = []

for i in range(2010,2022):
    busan_sample = busan[(busan['year'] == i) & (busan['temp'] >= 33)]
    result.append(len(busan_sample['day'].unique()))

print("부산 지역 매년 폭염 발생일 수" )
print(result)

sum = 0
for i in range(len(result)):
    sum += result[i]

mean = sum / len(result)


print("부산 지역 폭염 발생일 평균 : ",round(mean,2))

부산 지역 매년 폭염 발생일 수
[22, 19, 23, 28, 15, 14, 14, 21, 30, 17, 16, 15]
부산 지역 폭염 발생일 평균 :  19.5


# 대전

In [13]:
daejeon = df_daejeon[['region','year','month','day','hour','rainfall','temp','lightning','humidity','wind_speed','wind_direction','condition','HI','DI']]
daejeon

,region,year,month,day,hour,rainfall,temp,lightning,humidity,wind_speed,wind_direction,condition,HI,DI
0,중앙동,2010,6,1,0,0.0,17.400000,-1,44.0,53.0,390.0,1,16.344444,61.681440
1,중앙동,2010,6,1,1,0.0,18.700001,-1,39.0,53.0,390.0,1,17.643890,63.090071
2,중앙동,2010,6,1,2,0.0,19.900000,-1,37.0,53.0,390.0,1,18.911667,64.417370
3,중앙동,2010,6,1,3,0.0,20.600000,-1,35.0,53.0,390.0,1,19.629444,65.118900
4,중앙동,2010,6,1,4,0.0,21.700001,-1,33.0,53.0,390.0,1,20.787223,66.247391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8251387,목상동,2022,4,30,19,0.0,5.600000,-1,94.0,0.0,0.0,-1,4.670000,42.605360
8251388,목상동,2022,4,30,20,0.0,5.300000,-1,95.0,0.0,230.0,-1,4.366111,41.992650
8251389,목상동,2022,4,30,21,0.0,8.200000,-1,93.0,0.0,0.0,-1,7.503889,47.192740
8251390,목상동,2022,4,30,22,0.0,11.100000,-1,66.0,1.8,200.0,-1,9.988889,53.105740


In [30]:
result = []

for i in range(2010,2022):
    daejeon_sample = daejeon[(daejeon['year'] == i) & (daejeon['temp'] >= 33)]
    result.append(len(daejeon_sample['day'].unique()))

print("대전 지역 매년 폭염 발생일 수" )
print(result)

sum = 0
for i in range(len(result)):
    sum += result[i]

mean = sum / len(result)


print("대전 지역 폭염 발생일 평균 : ",round(mean,2))

대전 지역 매년 폭염 발생일 수
[19, 16, 21, 23, 7, 10, 25, 19, 31, 24, 18, 23]
대전 지역 폭염 발생일 평균 :  19.67


# 청주

In [18]:
cheongju = df_cheongju[['region','year','month','day','hour','rainfall','temp','lightning','humidity','wind_speed','wind_direction','condition','HI','DI']]
cheongju

,region,year,month,day,hour,rainfall,temp,lightning,humidity,wind_speed,wind_direction,condition,HI,DI
0,가경동,2010,6,1,0,0.0,17.299999,-1,41.0,53.0,390.0,1,16.156110,61.472069
1,가경동,2010,6,1,1,0.0,19.000000,-1,38.0,53.0,390.0,1,17.947778,63.403800
2,가경동,2010,6,1,2,0.0,20.299999,-1,35.0,53.0,390.0,1,19.299443,64.771949
3,가경동,2010,6,1,3,0.0,21.600000,-1,33.0,53.0,390.0,1,20.677222,66.133720
4,가경동,2010,6,1,4,0.0,22.900000,-1,35.0,53.0,390.0,1,22.159444,67.778850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4491259,현도면,2022,4,30,19,0.0,6.900000,-1,92.0,0.5,125.0,-1,6.047778,45.017520
4491260,현도면,2022,4,30,20,0.0,6.600000,-1,94.0,0.6,182.0,-1,5.770000,44.345960
4491261,현도면,2022,4,30,21,0.0,8.300000,-1,91.0,0.8,154.0,-1,7.561667,47.487470
4491262,현도면,2022,4,30,22,0.0,10.800000,-1,74.0,1.1,211.0,-1,9.867778,52.378080


In [29]:
result = []

for i in range(2010,2022):
    cheongju_sample = cheongju[(cheongju['year'] == i) & (cheongju['temp'] >= 33)]
    result.append(len(cheongju_sample['day'].unique()))

print("청주 지역 매년 폭염 발생일 수" )
print(result)

sum = 0
for i in range(len(result)):
    sum += result[i]

mean = sum / len(result)


print("청주 지역 폭염 발생일 평균 : ",round(mean,2))

청주 지역 매년 폭염 발생일 수
[20, 16, 18, 23, 8, 11, 24, 15, 31, 20, 16, 22]
청주 지역 폭염 발생일 평균 :  18.67


# 춘천

In [21]:
chuncheon = df_chuncheon[['region','year','month','day','hour','rainfall','temp','lightning','humidity','wind_speed','wind_direction','condition','HI','DI']]
chuncheon

,region,year,month,day,hour,rainfall,temp,lightning,humidity,wind_speed,wind_direction,condition,HI,DI
0,강남동,2010,6,1,0,0.0,14.300000,-1,47.0,52.799999,389.0,1,13.012778,57.815790
1,강남동,2010,6,1,1,0.0,17.200001,-1,37.0,52.799999,389.0,1,15.941668,61.241361
2,강남동,2010,6,1,2,0.0,20.299999,-1,29.0,52.799999,389.0,1,19.142777,64.424129
3,강남동,2010,6,1,3,0.0,21.700001,-1,25.0,52.799999,389.0,1,20.578334,65.672751
4,강남동,2010,6,1,4,0.0,23.700001,-1,23.0,52.799999,389.0,1,22.726112,67.604491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2715643,후평3동,2022,4,30,19,0.0,4.200000,-1,97.0,0.400000,78.0,-1,4.200000,39.864260
2715644,후평3동,2022,4,30,20,0.0,3.900000,-1,98.0,0.500000,317.0,-1,3.900000,39.228780
2715645,후평3동,2022,4,30,21,0.0,5.600000,-1,95.0,0.900000,318.0,-1,4.696111,42.517800
2715646,후평3동,2022,4,30,22,0.0,7.700000,-1,85.0,1.100000,316.0,-1,6.745000,46.861550


In [28]:
result = []

for i in range(2010,2022):
    chuncheon_sample = chuncheon[(chuncheon['year'] == i) & (chuncheon['temp'] >= 33)]
    result.append(len(chuncheon_sample['day'].unique()))

print("춘천 지역 매년 폭염 발생일 수" )
print(result)

sum = 0
for i in range(len(result)):
    sum += result[i]

mean = sum / len(result)


print("춘천 지역 폭염 발생일 평균 : ",round(mean,2))

춘천 지역 매년 폭염 발생일 수
[15, 14, 16, 10, 11, 9, 20, 14, 31, 17, 10, 23]
춘천 지역 폭염 발생일 평균 :  15.83
